# BOM data workshop excercise #2

 Creates an animation of the pressure-level wind data from BARRA-TA and BARRA-R sample data using netCDF4 and matplotlib.
 Wind vectors over 37 pressure levels.

 Code written for illustrating BARRA pressure-level data.

 Chun-Hsu Su, 19 January 2018
 The Bureau of Meteorology



In [2]:
import os, sys
import numpy as np
from netCDF4 import Dataset, num2date, date2num
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import cartopy.crs as ccrs
from matplotlib import cm
from mpl_toolkits.basemap import Basemap

## Define methods needed to render the animation

In [3]:
def get_uv(j):
	"""
	Returns the lon,lat,pres,u,v values for a given pressure level
	identified by index j.
	"""
	_,a,b = modwind['uwind'].shape
	c = max(a,b)
	ivl = c/30
	x = modwind['lon'][::ivl]
	y = modwind['lat'][::ivl]
	u = modwind['uwind'][j,::ivl,::ivl]
	v = modwind['vwind'][j,::ivl,::ivl]
	p = modwind['prs'][j]
	return x,y,p,u,v

In [4]:
def update_quiver(j,ax,fig):
	"""
	Update the figure with the wind field at pressure level j.
	"""
	print("Plotting pressure level {:}".format(j))
	x,y,p,u,v = get_uv(j)
	Q.set_UVC(u,v)
	ax.set_title('Pressure level: {:3.3f} hPa'.format(p),fontsize=14)
	return Q

In [5]:
def init_quiver():
	"""
	Initialise the figure with the wind field at pressure level j = 0.
	"""
	global Q
	x,y,p,u,v = get_uv(0)
	Q = ax.quiver(x,y,u,v)
	ax.set_title('Pressure level: {:3.3f} hPa'.format(p),fontsize=14)
	#ax.set_xlabel('Longitude')
	#ax.set_ylabel('Latitude')
	return Q

## Main script

### Specify the data inputs

In [6]:
# datetime of file to animate, choose 2015/02/17 00 - 18 for sample data 
dtFile = datetime(2015,2,7,12,0)

In [7]:
# model string, choose BARRA_R or BARRA_TA for sample data
model = 'BARRA_R'

In [8]:
# define forecast hour to animate, 1-6 for SY and TA or 1-9 for R
forecastHour = 6

In [9]:
# basetime
dtBasetime = dtFile - timedelta(hours = forecastHour)

In [10]:
### Find the data

In [11]:
# Where are your sample data downloaded to?
dataDirRoot = '/g/data/ma05/sample'
dataDirTemplate = os.path.join(dataDirRoot, '{model}/v1')

In [12]:
# iso 1806 time format string
isoFormat = '%Y%m%dT%H%MZ'

In [14]:
# file names have a specific structure eg:  BARRA_R/v1/forecast/spec/temp_scrn/2015/12/temp_scrn-fc-spec-PT10M-BARRA_R-v1-20151208T1800Z.nc
fnTemplate = os.path.join(dataDirTemplate, 'forecast', '{stream}', '{var}',\
                          '{year:04d}', '{month:02d}', '{var}-fc-{stream}-PT1H-{model}-v1-{basetimeStr}.nc')

In [ ]:
uFilename = fntemplate.format(stream='prs', var='wnd_ucmp', year=dtBasetime.year, month = dtBasetime.month, \
                             basetimeStr = dtBasetime.strf)